# Cleaning the dataset, gathering geometry 

### This notebook is to clean the csv containing property listings from Daft.ie and obtain their coordinates


In [220]:
# !pip install geopandas

In [221]:
# !pip install python-dotenv

In [222]:
# !pip install nominatim

In [223]:
# !pip install geopy

In [224]:
import pandas as pd
import requests
import re
import json
import geopandas as gpd
import nominatim
import os
import time
import argparse
from datetime import date
from pprint import pprint
from typing import Dict
from dotenv import load_dotenv
from nominatim import Nominatim
load_dotenv()

False

In [225]:
# today = date.today()
# year = today.year
# month = today.month
# day = today.day

df = pd.read_csv("./DOHMH-CSV-last.csv")

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Record ID             11093 non-null  object 
 1   SchoolName            11093 non-null  object 
 2   Number                11093 non-null  object 
 3   Street                11093 non-null  object 
 4   City                  10657 non-null  object 
 5   State                 11093 non-null  object 
 6   Borough               11093 non-null  object 
 7   ZipCode               10942 non-null  float64
 8   LastInspection        11093 non-null  object 
 9   Permittee             11093 non-null  object 
 10  InspectionDate        11093 non-null  object 
 11  PTET                  11093 non-null  int64  
 12  Site Type             11093 non-null  object 
 13  Level                 7974 non-null   object 
 14  Code                  7974 non-null   object 
 15  ViolationDescriptio

In [227]:
df.head(5)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,ViolationDescription,LATITUDE,LONGITUDE,COMMUNITYBOARD,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,NY,Brooklyn,11228.0,02/02/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,40.624125,-74.018826,310.0,43.0,20600.0,3150481.0,3059730001,BK30,3
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,NY,Brooklyn,NaN,02/15/2023,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,Toilet facility not maintained and provided wi...,0.000000,0.000000,NaN,NaN,NaN,NaN,3,NaN,3
2,40542116,P.S. 135 SHELDON A. BROOKNER,684,LINDEN BOULEVARD,Brooklyn,NY,Brooklyn,11203.0,01/23/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,Food Protection Certificate (FPC) not held by ...,40.653633,-73.933114,317.0,41.0,87000.0,3102005.0,3046730001,BK91,3
3,40542900,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,1300,BOYNTON AVENUE,Bronx,NY,Bronx,10472.0,03/09/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,Evidence of mice or live mice in establishment...,40.831594,-73.878911,209.0,18.0,5600.0,2027459.0,2038640001,BX08,2
4,40541560,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",34,WEST 118 STREET,New York,NY,Manhattan,10026.0,01/31/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,Live roaches in facility's food or non-food area.,40.802464,-73.946715,110.0,9.0,19000.0,1051434.0,1016010001,MN11,1


In [228]:
x = df["LATITUDE"] == 0
x.value_counts()

LATITUDE
False    10942
True       151
Name: count, dtype: int64

In [229]:
df["ZipCode"].isnull().value_counts()

ZipCode
False    10942
True       151
Name: count, dtype: int64

In [230]:
df[["Number", "Street", "City", "ZipCode"]].head(2)

,Number,Street,City,ZipCode
0,7805,7 AVENUE,Brooklyn,11228.0
1,141315,45 STREET,BROOKLYN,NaN


In [231]:
df["cleaned_city"] = df["City"].str.capitalize()
df["cleaned_city"]

0                Brooklyn
1                Brooklyn
2                Brooklyn
3                   Bronx
4                New york
               ...       
11088          Ozone park
11089      Queens village
11090            New york
11091    Long island city
11092            Brooklyn
Name: cleaned_city, Length: 11093, dtype: object

In [232]:
df["Borough"] = df["Borough"].str.capitalize()
df["Borough"]

0         Brooklyn
1         Brooklyn
2         Brooklyn
3            Bronx
4        Manhattan
           ...    
11088       Queens
11089       Queens
11090    Manhattan
11091       Queens
11092     Brooklyn
Name: Borough, Length: 11093, dtype: object

In [233]:
df["State"] = df["State"].str.replace("NY", "New York")
df["State"]

0        New York
1        New York
2        New York
3        New York
4        New York
           ...   
11088    New York
11089    New York
11090    New York
11091    New York
11092    New York
Name: State, Length: 11093, dtype: object

In [234]:
def find_last_digit(input_string):
    # Use a regular expression to find all digits in the input string
    digits = re.findall(r'\d', input_string)
    if not digits:
        return input_string
    else:
        matches = re.search(r'(.*)(\d)(.*)', input_string)
        beginning = matches.group(1)
        last_digit = matches.group(2)
        ending = matches.group(3)
        if (last_digit == "1"):
            new_last_digit = last_digit + "st"
        elif (last_digit == "2"):
            new_last_digit = last_digit + "nd"
        elif (last_digit == "3"):
            new_last_digit = last_digit + "rd"
        else:
            new_last_digit = last_digit + "th"

        return beginning + new_last_digit + ending

In [235]:
find_last_digit("30 Avenue")

'30th Avenue'

In [236]:
find_last_digit("31 Avenue")

'31st Avenue'

In [237]:
find_last_digit("32 Avenue")

'32nd Avenue'

In [238]:
find_last_digit("33 Avenue")

'33rd Avenue'

In [239]:
find_last_digit("34 Avenue")

'34th Avenue'

In [240]:
find_last_digit("37 Avenue")

'37th Avenue'

In [241]:
df["full_street"] = df["Street"].apply(lambda x: find_last_digit(x))
df

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,LONGITUDE,COMMUNITYBOARD,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,New York,Brooklyn,11228.0,02/02/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-74.018826,310.0,43.0,20600.0,3150481.0,3059730001,BK30,3,Brooklyn,7th AVENUE
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,New York,Brooklyn,NaN,02/15/2023,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,0.000000,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,45th STREET
2,40542116,P.S. 135 SHELDON A. BROOKNER,684,LINDEN BOULEVARD,Brooklyn,New York,Brooklyn,11203.0,01/23/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.933114,317.0,41.0,87000.0,3102005.0,3046730001,BK91,3,Brooklyn,LINDEN BOULEVARD
3,40542900,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,1300,BOYNTON AVENUE,Bronx,New York,Bronx,10472.0,03/09/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.878911,209.0,18.0,5600.0,2027459.0,2038640001,BX08,2,Bronx,BOYNTON AVENUE
4,40541560,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",34,WEST 118 STREET,New York,New York,Manhattan,10026.0,01/31/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.946715,110.0,9.0,19000.0,1051434.0,1016010001,MN11,1,New york,WEST 118th STREET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11088,40541399,P.S. 064 JOSEPH P. ADDABBO,8201,101 AVENUE,OZONE PARK,New York,Queens,11416.0,02/27/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.856918,409.0,32.0,4200.0,4188214.0,4090530014,QN53,4,Ozone park,101st AVENUE
11089,40541012,"P.S. Q177, MARTIN VAN BUREN HIGH SCHOOL, BUSIN...",23017,HILLSIDE AVENUE,QUEENS VILLAGE,New York,Queens,11427.0,01/12/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.738755,413.0,23.0,130100.0,4166101.0,4079300002,QN43,4,Queens village,HILLSIDE AVENUE
11090,40655565,THE DALTON SCHOOL,108,EAST 89 STREET,New York,New York,Manhattan,10128.0,10/25/2022,FLIK INTERNATIONAL CORP,...,-73.955228,108.0,4.0,14802.0,1048100.0,1015170062,MN40,1,New york,EAST 89th STREET
11091,41442488,GROWING UP GREEN CHARTER SCHOOL,3937,28 STREET,LONG ISLAND CITY,New York,Queens,11101.0,10/06/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,-73.935703,401.0,26.0,3100.0,4436781.0,4003980001,QN68,4,Long island city,28th STREET


In [242]:
df["address"] = df.apply(lambda row: f'{row["Number"]} {row["full_street"]}, {row["cleaned_city"]}, {row["State"]}', axis=1)
df["address"]

0                    7805 7th AVENUE, Brooklyn, New York
1                 141315 45th STREET, Brooklyn, New York
2               684 LINDEN BOULEVARD, Brooklyn, New York
3                   1300 BOYNTON AVENUE, Bronx, New York
4              34 WEST  118th STREET, New york, New York
                              ...                       
11088            8201 101st AVENUE, Ozone park, New York
11089    23017 HILLSIDE AVENUE, Queens village, New York
11090         108 EAST   89th STREET, New york, New York
11091       3937 28th STREET, Long island city, New York
11092               5801 16th AVENUE, Brooklyn, New York
Name: address, Length: 11093, dtype: object

In [243]:
# subset of df with coordinates - no action needed

has_coords_df = df[~((df["LATITUDE"] == 0) | (df["LONGITUDE"] == 0))].copy()
has_coords_df

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,COMMUNITYBOARD,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street,address
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,New York,Brooklyn,11228.0,02/02/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,310.0,43.0,20600.0,3150481.0,3059730001,BK30,3,Brooklyn,7th AVENUE,"7805 7th AVENUE, Brooklyn, New York"
2,40542116,P.S. 135 SHELDON A. BROOKNER,684,LINDEN BOULEVARD,Brooklyn,New York,Brooklyn,11203.0,01/23/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,317.0,41.0,87000.0,3102005.0,3046730001,BK91,3,Brooklyn,LINDEN BOULEVARD,"684 LINDEN BOULEVARD, Brooklyn, New York"
3,40542900,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,1300,BOYNTON AVENUE,Bronx,New York,Bronx,10472.0,03/09/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,209.0,18.0,5600.0,2027459.0,2038640001,BX08,2,Bronx,BOYNTON AVENUE,"1300 BOYNTON AVENUE, Bronx, New York"
4,40541560,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",34,WEST 118 STREET,New York,New York,Manhattan,10026.0,01/31/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,110.0,9.0,19000.0,1051434.0,1016010001,MN11,1,New york,WEST 118th STREET,"34 WEST 118th STREET, New york, New York"
5,40542147,"P.S. 244 RICHARD R. GREEN, BROOKLYN SCIENCE AN...",5400,TILDEN AVENUE,Brooklyn,New York,Brooklyn,11203.0,01/23/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,317.0,45.0,94600.0,3104513.0,3047390001,BK96,3,Brooklyn,TILDEN AVENUE,"5400 TILDEN AVENUE, Brooklyn, New York"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11088,40541399,P.S. 064 JOSEPH P. ADDABBO,8201,101 AVENUE,OZONE PARK,New York,Queens,11416.0,02/27/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,409.0,32.0,4200.0,4188214.0,4090530014,QN53,4,Ozone park,101st AVENUE,"8201 101st AVENUE, Ozone park, New York"
11089,40541012,"P.S. Q177, MARTIN VAN BUREN HIGH SCHOOL, BUSIN...",23017,HILLSIDE AVENUE,QUEENS VILLAGE,New York,Queens,11427.0,01/12/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,413.0,23.0,130100.0,4166101.0,4079300002,QN43,4,Queens village,HILLSIDE AVENUE,"23017 HILLSIDE AVENUE, Queens village, New York"
11090,40655565,THE DALTON SCHOOL,108,EAST 89 STREET,New York,New York,Manhattan,10128.0,10/25/2022,FLIK INTERNATIONAL CORP,...,108.0,4.0,14802.0,1048100.0,1015170062,MN40,1,New york,EAST 89th STREET,"108 EAST 89th STREET, New york, New York"
11091,41442488,GROWING UP GREEN CHARTER SCHOOL,3937,28 STREET,LONG ISLAND CITY,New York,Queens,11101.0,10/06/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,401.0,26.0,3100.0,4436781.0,4003980001,QN68,4,Long island city,28th STREET,"3937 28th STREET, Long island city, New York"


In [244]:
# subset of df missing coordinates - run function to get coords

missing_coords_df = df[((df["LATITUDE"] == 0) | (df["LONGITUDE"] == 0))].copy()
missing_coords_df.head(5)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,COMMUNITYBOARD,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street,address
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,New York,Brooklyn,NaN,02/15/2023,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,45th STREET,"141315 45th STREET, Brooklyn, New York"
136,40542048,EAST BROOKLYN COMMUNITY HIGH SCHOOL,9517,KINGS HGWAY,BROOKLYN,New York,Brooklyn,NaN,11/07/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,KINGS HGWAY,"9517 KINGS HGWAY, Brooklyn, New York"
177,40541138,"P.S. 030 HERNANDEZ/HUGHES, HARLEM SUCCESS ACAD...",144176,EAST 128 STREET,NEW YORK,New York,Manhattan,NaN,02/15/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,NaN,1,NaN,1,New york,EAST 128th STREET,"144176 EAST 128th STREET, New york, New York"
237,40542048,EAST BROOKLYN COMMUNITY HIGH SCHOOL,9517,KINGS HGWAY,BROOKLYN,New York,Brooklyn,NaN,11/07/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,KINGS HGWAY,"9517 KINGS HGWAY, Brooklyn, New York"
329,41175519,UNITED TALMUDICAL ACADEMY,212,WILLIAMSBURG STREET,BROOKLYN,New York,Brooklyn,NaN,01/12/2023,TALMUDICAL ACAD UNITED,...,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,WILLIAMSBURG STREET,"212 WILLIAMSBURG STREET, Brooklyn, New York"


In [245]:
query_params = {
    "namedetails": 1,
    "polygon_geojson": 1,
    "hierarchy": 1,
}

def fetch_osm_search(query: str, params: Dict[str, int]) -> dict:
    NOMINATIM_API_URL = "https://nominatim.openstreetmap.org"
    NOMINATIM_SEARCH_ENDPOINT = f"{NOMINATIM_API_URL}/search"

    url_query = query.replace(" ", "%20")
    
    params_query = "&".join(f"{param_name}={param_value}" for param_name, param_value in params.items())
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?q={url_query}&{params_query}&format=json"
    print(request_url)

    response = requests.get(request_url)
    response.raise_for_status()
    return response.json()

In [246]:
test = fetch_osm_search("2365 38th Street, queens, new york", params=query_params)
w = test[0]['lat']
print(w)

https://nominatim.openstreetmap.org/search?q=2365%2038th%20Street,%20queens,%20new%20york&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
40.7410495


In [247]:
# missing_coords_df.apply(lambda x: fetch_osm_search(x["address"], params=query_params), axis=1)

In [128]:
for i,row in enumerate (missing_coords_df["address"]):
    coordinates = fetch_osm_search(row, params=query_params)
    try:
        coordinates_latitude = coordinates[0]['lat']
        print(coordinates_latitude)
#         row["LATITUDE"] = coordinates_latitude
#         missing_coords_df["LATITUDE"].loc[i] = coordinates_latitude
#         missing_coords_df.loc[i,"LATITUDE"] = coordinates_latitude
#         row["LATITUDE"].loc[i] = coordinates_latitude
        
        coordinates_longitude = coordinates[0]['lon']
        print(coordinates_longitude)
                
#         new_coordinates.loc[index] = f"{coordinates_latitude},{coordinates_longitude}"
        
        time.sleep(2)
    except:
        pass
        time.sleep(2)

https://nominatim.openstreetmap.org/search?q=141315%2045th%20STREET,%20Brooklyn,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
https://nominatim.openstreetmap.org/search?q=9517%20KINGS%20HGWAY,%20Brooklyn,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
https://nominatim.openstreetmap.org/search?q=144176%20EAST%20%20128th%20STREET,%20New%20york,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
https://nominatim.openstreetmap.org/search?q=9517%20KINGS%20HGWAY,%20Brooklyn,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json


KeyboardInterrupt: 

In [197]:
# for item in missing_coords_df["LATITUDE"]:
#     if item == 0:
#         try:
#             response = missing_coords_df.apply(lambda x: fetch_osm_search(x["address"], params=query_params), axis=1)
#             respone_latitude = response[0]['lat']
#             respone_longitude = response[0]['lon']
#             missing_coords_df["coordinates"] = (response_latitude + "," + response_longitude)
#         except:
#             pass

In [287]:
def getLat (address):
    print('------')
    coordinates = fetch_osm_search(address, params=query_params)
    try:
        coordinates_latitude = coordinates[0]['lat']
        print(coordinates_latitude)
        lat = coordinates_latitude
        time.sleep(2)
        return lat
    except:
        pass
        time.sleep(2)
        return 0 

In [288]:
getLat('7805 7th Avenue')

------
https://nominatim.openstreetmap.org/search?q=7805%207th%20Avenue&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
-34.0418126


'-34.0418126'

In [291]:
getLat(missing_coords_df["address"])

------
https://nominatim.openstreetmap.org/search?q=1               141315 45th STREET, Brooklyn, New York
136               9517 KINGS HGWAY, Brooklyn, New York
177      144176 EAST  128th STREET, New york, New York
237               9517 KINGS HGWAY, Brooklyn, New York
329        212 WILLIAMSBURG STREET, Brooklyn, New York
                             ...                      
10822    200214 WEST  135th STREET, New york, New York
10847      212 WILLIAMSBURG STREET, Brooklyn, New York
10914         141147 EAST BROADWAY, New york, New York
11008    144176 EAST  128th STREET, New york, New York
11015         141147 EAST BROADWAY, New york, New York
Name: address, Length: 151, dtype: object&namedetails=1&polygon_geojson=1&hierarchy=1&format=json


0

In [292]:
missing_coords_df["LATITUDE"]= getLat(missing_coords_df["address"])

------
https://nominatim.openstreetmap.org/search?q=1               141315 45th STREET, Brooklyn, New York
136               9517 KINGS HGWAY, Brooklyn, New York
177      144176 EAST  128th STREET, New york, New York
237               9517 KINGS HGWAY, Brooklyn, New York
329        212 WILLIAMSBURG STREET, Brooklyn, New York
                             ...                      
10822    200214 WEST  135th STREET, New york, New York
10847      212 WILLIAMSBURG STREET, Brooklyn, New York
10914         141147 EAST BROADWAY, New york, New York
11008    144176 EAST  128th STREET, New york, New York
11015         141147 EAST BROADWAY, New york, New York
Name: address, Length: 151, dtype: object&namedetails=1&polygon_geojson=1&hierarchy=1&format=json


In [293]:
missing_coords_df["LATITUDE"].value_counts().unique()

array([151], dtype=int64)

In [294]:
missing_coords_df["LATITUDE"].head(50)

1       0
136     0
177     0
237     0
329     0
413     0
414     0
467     0
537     0
550     0
552     0
579     0
689     0
795     0
807     0
943     0
954     0
1211    0
1460    0
1510    0
1517    0
1544    0
1575    0
1667    0
1701    0
1790    0
1835    0
1882    0
1891    0
1893    0
1981    0
1989    0
2288    0
2291    0
2503    0
2511    0
2551    0
2597    0
2608    0
2662    0
2677    0
2775    0
2783    0
2919    0
2945    0
2957    0
2979    0
3152    0
3153    0
3159    0
Name: LATITUDE, dtype: int64

In [204]:
missing_coords_df.apply(lambda x: fetch_osm_search(x["address"], params=query_params), axis=1)

https://nominatim.openstreetmap.org/search?q=141315%2045th%20STREET,%20Brooklyn,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
https://nominatim.openstreetmap.org/search?q=9517%20KINGS%20HGWAY,%20Brooklyn,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json
https://nominatim.openstreetmap.org/search?q=144176%20EAST%20%20128th%20STREET,%20New%20york,%20New%20York&namedetails=1&polygon_geojson=1&hierarchy=1&format=json


KeyboardInterrupt: 

In [ ]:
# new_coordinates = []

# for item in missing_coords_df["address"]:
#     coordinates = fetch_osm_search(item, params=query_params)
#     # dict = {}
#     try:
#         coordinates_latitude = coordinates[0]['lat']
#         # dict["lat"] =
#         print(coordinates_latitude)
#         coordinates_longitude = coordinates[0]['lon']
#         print(coordinates_longitude)
#         new_coordinates.loc[index] = f"{coordinates_latitude},{coordinates_longitude}"
#         time.sleep(2)
#     except:
#         pass
#         time.sleep(2)

In [ ]:
for i,row in enumerate (missing_coords_df):
    coordinates = fetch_osm_search(row[i].address, params=query_params)
    try:
        coordinates_latitude = coordinates[0]['lat']
        print(coordinates_latitude)
        row["LATITUDE"] = coordinates_latitude
        # row["LATITUDE"].loc[i] = coordinates_latitude
        
        # coordinates_longitude = coordinates[0]['lon']
        # print(coordinates_longitude)
        # new_coordinates.loc[index] = f"{coordinates_latitude},{coordinates_longitude}"
        time.sleep(2)
    except:
        pass
        time.sleep(2)

In [55]:
# from geopy.geocoders import Nominatim

# for item in missing_coords_df["address"]: 
 
#     # calling the Nominatim tool
#     loc = Nominatim(user_agent="GetLoc")
 
#     # entering the location name
#     getLoc = loc.geocode("Gosainganj Lucknow")
 
#     # printing address
#     print(getLoc.address)
 
#     # printing latitude and longitude
#     print("Latitude = ", getLoc.latitude, "\n")
#     print("Longitude = ", getLoc.longitude)

In [69]:
missing_coords_df.head(50)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street,address,coordinates
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,New York,Brooklyn,NaN,02/15/2023,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,NaN,NaN,NaN,3,NaN,3,Brooklyn,45th STREET,"141315 45th STREET, Brooklyn, New York",
136,40542048,EAST BROOKLYN COMMUNITY HIGH SCHOOL,9517,KINGS HGWAY,BROOKLYN,New York,Brooklyn,NaN,11/07/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,3,NaN,3,Brooklyn,KINGS HGWAY,"9517 KINGS HGWAY, Brooklyn, New York",
177,40541138,"P.S. 030 HERNANDEZ/HUGHES, HARLEM SUCCESS ACAD...",144176,EAST 128 STREET,NEW YORK,New York,Manhattan,NaN,02/15/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,1,NaN,1,New york,EAST 128th STREET,"144176 EAST 128th STREET, New york, New York",
237,40542048,EAST BROOKLYN COMMUNITY HIGH SCHOOL,9517,KINGS HGWAY,BROOKLYN,New York,Brooklyn,NaN,11/07/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,3,NaN,3,Brooklyn,KINGS HGWAY,"9517 KINGS HGWAY, Brooklyn, New York",
329,41175519,UNITED TALMUDICAL ACADEMY,212,WILLIAMSBURG STREET,BROOKLYN,New York,Brooklyn,NaN,01/12/2023,TALMUDICAL ACAD UNITED,...,NaN,NaN,NaN,3,NaN,3,Brooklyn,WILLIAMSBURG STREET,"212 WILLIAMSBURG STREET, Brooklyn, New York",
413,40541330,"P.S. Q004 AT Q138, P.S./ M.S.138 SUNRISE",25311,WELLER AVENUE,QUEENS,New York,Queens,NaN,02/07/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,4,NaN,4,Queens,WELLER AVENUE,"25311 WELLER AVENUE, Queens, New York",
414,40998725,GREATER TOMORROW ELEM/MDDLE SCHOOL/ ATLAH WORL...,3638,WEST 123 STREET,NEW YORK,New York,Manhattan,NaN,01/05/2023,"BETHELITE COMMUNITY CHURCH, INC.",...,NaN,NaN,NaN,1,NaN,1,New york,WEST 123rd STREET,"3638 WEST 123rd STREET, New york, New York",
467,50126133,NEW YORK FRENCH AMERCAN CHARTER SCHOOL,2116,ADAM CLAYTON POWELL JR BOULEVARD,New York,New York,Manhattan,NaN,02/09/2023,NEW YORK FRENCH AMERCAN CHARTER SCHOOL,...,NaN,NaN,NaN,1,NaN,1,New york,ADAM CLAYTON POWELL JR BOULEVARD,"2116 ADAM CLAYTON POWELL JR BOULEVARD, New yor...",
537,40541239,P.S. 046 ARTHUR TAPPAN,2987,FREDERICK DOUGLASS BOULEVARD,New York,New York,Manhattan,NaN,10/03/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,1,NaN,1,New york,FREDERICK DOUGLASS BOULEVARD,"2987 FREDERICK DOUGLASS BOULEVARD, New york, N...",
550,40543508,P.S. 108 PHILIP J. ABINANTI,1166,NEIL AVENUE,BRONX,New York,Bronx,NaN,11/21/2022,NEW YORK CITY DEPARTMENT OF EDUCATION,...,NaN,NaN,NaN,2,NaN,2,Bronx,NEIL AVENUE,"1166 NEIL AVENUE, Bronx, New York",


In [ ]:
missing_coords_df.apply(lambda x: fetch_osm_search(x["address"], params=query_params), axis=1)

In [ ]:
addresses = missing_coords_df["address"].tolist()

In [40]:
coordinates_df = []

#looping through the locations

for ax in addresses:

    #     url = 'https://maps.googleapis.com/maps/api/geocode/json'
#     keys = {'address': place, 'key': api_key}
#     r = requests.get(url,params=keys)
    fetch_osm_search(ax)
    
    lat = result_dic['results'][0]['geometry']['location']['lat']
    long = result_dic['results'][0]['geometry']['location']['lng']
    coords = []
    coords.append(long)
    coords.append(lat)
    mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords, 'latitude': lat, 'longitude': long}
    coordinates_df.append(mygeometry)
    print(mygeometry)
    time.sleep(0.5)

### Data Analysis.

In [248]:
df.head(5)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,COMMUNITYBOARD,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street,address
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,New York,Brooklyn,11228.0,02/02/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,310.0,43.0,20600.0,3150481.0,3059730001,BK30,3,Brooklyn,7th AVENUE,"7805 7th AVENUE, Brooklyn, New York"
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,New York,Brooklyn,NaN,02/15/2023,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,NaN,NaN,NaN,NaN,3,NaN,3,Brooklyn,45th STREET,"141315 45th STREET, Brooklyn, New York"
2,40542116,P.S. 135 SHELDON A. BROOKNER,684,LINDEN BOULEVARD,Brooklyn,New York,Brooklyn,11203.0,01/23/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,317.0,41.0,87000.0,3102005.0,3046730001,BK91,3,Brooklyn,LINDEN BOULEVARD,"684 LINDEN BOULEVARD, Brooklyn, New York"
3,40542900,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,1300,BOYNTON AVENUE,Bronx,New York,Bronx,10472.0,03/09/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,209.0,18.0,5600.0,2027459.0,2038640001,BX08,2,Bronx,BOYNTON AVENUE,"1300 BOYNTON AVENUE, Bronx, New York"
4,40541560,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",34,WEST 118 STREET,New York,New York,Manhattan,10026.0,01/31/2023,NEW YORK CITY DEPARTMENT OF EDUCATION,...,110.0,9.0,19000.0,1051434.0,1016010001,MN11,1,New york,WEST 118th STREET,"34 WEST 118th STREET, New york, New York"


In [249]:
df.dtypes

Record ID                object
SchoolName               object
Number                   object
Street                   object
City                     object
State                    object
Borough                  object
ZipCode                 float64
LastInspection           object
Permittee                object
InspectionDate           object
PTET                      int64
Site Type                object
Level                    object
Code                     object
ViolationDescription     object
LATITUDE                float64
LONGITUDE               float64
COMMUNITYBOARD          float64
COUNCILDISTRICT         float64
CENSUSTRACT             float64
BIN                     float64
BBL                       int64
NTA                      object
BOROCODE                  int64
cleaned_city             object
full_street              object
address                  object
dtype: object

In [250]:
df['LastInspection'] = pd.to_datetime(df['LastInspection'])
df['LastInspection']

0       2023-02-02
1       2023-02-15
2       2023-01-23
3       2023-03-09
4       2023-01-31
           ...    
11088   2023-02-27
11089   2023-01-12
11090   2022-10-25
11091   2022-10-06
11092   2023-01-18
Name: LastInspection, Length: 11093, dtype: datetime64[ns]

In [251]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])
df['InspectionDate']

0       2022-07-28
1       2022-03-01
2       2022-11-17
3       2023-03-09
4       2022-08-01
           ...    
11088   2023-02-27
11089   2023-01-12
11090   2022-10-25
11091   2021-04-21
11092   2022-02-23
Name: InspectionDate, Length: 11093, dtype: datetime64[ns]

In [252]:
df['Year'] = df['InspectionDate'].dt.strftime('%Y')
df.head(5)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,Permittee,...,COUNCILDISTRICT,CENSUSTRACT,BIN,BBL,NTA,BOROCODE,cleaned_city,full_street,address,Year
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,New York,Brooklyn,11228.0,2023-02-02,NEW YORK CITY DEPARTMENT OF EDUCATION,...,43.0,20600.0,3150481.0,3059730001,BK30,3,Brooklyn,7th AVENUE,"7805 7th AVENUE, Brooklyn, New York",2022
1,40736086,BAIS YITZHAK SCHOOL,141315,45 STREET,BROOKLYN,New York,Brooklyn,NaN,2023-02-15,CONGREGATION KHAL CHASIDEI SKWERE - BAIS YITZA...,...,NaN,NaN,NaN,3,NaN,3,Brooklyn,45th STREET,"141315 45th STREET, Brooklyn, New York",2022
2,40542116,P.S. 135 SHELDON A. BROOKNER,684,LINDEN BOULEVARD,Brooklyn,New York,Brooklyn,11203.0,2023-01-23,NEW YORK CITY DEPARTMENT OF EDUCATION,...,41.0,87000.0,3102005.0,3046730001,BK91,3,Brooklyn,LINDEN BOULEVARD,"684 LINDEN BOULEVARD, Brooklyn, New York",2022
3,40542900,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONR...,1300,BOYNTON AVENUE,Bronx,New York,Bronx,10472.0,2023-03-09,NEW YORK CITY DEPARTMENT OF EDUCATION,...,18.0,5600.0,2027459.0,2038640001,BX08,2,Bronx,BOYNTON AVENUE,"1300 BOYNTON AVENUE, Bronx, New York",2023
4,40541560,"PS811M @ PS 149M, SUCCESS ACADEMY CHARTER-HARL...",34,WEST 118 STREET,New York,New York,Manhattan,10026.0,2023-01-31,NEW YORK CITY DEPARTMENT OF EDUCATION,...,9.0,19000.0,1051434.0,1016010001,MN11,1,New york,WEST 118th STREET,"34 WEST 118th STREET, New york, New York",2022


In [253]:
last_insp_year = df['Year'].value_counts()
last_insp_year

Year
2022    5917
2021    2604
2023    1995
2020     577
Name: count, dtype: int64

In [254]:
year_code = df.groupby('Year').Code.value_counts().reset_index()
year_codes = year_code.sort_values(by=['Year', 'count'], ascending=False)

In [255]:
year_codes

,Year,Code,count
120,2023,10F,381
121,2023,08A,205
122,2023,04L,191
123,2023,10B,162
124,2023,04A,64
...,...,...,...
11,2020,06D,1
12,2020,10J,1
13,2020,04N,1
14,2020,18B,1


In [279]:
year_codes_2023 = year_codes[year_codes['Year'] == '2023']
year_codes_2023.head(7)

,Year,Code,count
120,2023,10F,381
121,2023,08A,205
122,2023,04L,191
123,2023,10B,162
124,2023,04A,64
125,2023,06C,63
126,2023,09A,41


In [280]:
year_codes_2023.to_csv('year_codes_2023.csv') 

In [281]:
year_codes_2022 = year_codes[year_codes['Year'] == '2022']
year_codes_2022.head(7)

,Year,Code,count
57,2022,10F,1159
58,2022,08A,493
59,2022,10B,423
60,2022,04L,400
61,2022,04A,233
62,2022,04N,142
63,2022,06C,136


In [282]:
year_codes_2022.to_csv('year_codes_2022.csv') 

In [283]:
year_codes_2021 = year_codes[year_codes['Year'] == '2021']
year_codes_2021.head(7)

,Year,Code,count
16,2021,10F,541
17,2021,10B,225
18,2021,08A,148
19,2021,04L,107
20,2021,04A,67
21,2021,04N,45
22,2021,02G,44


In [284]:
year_codes_2021.to_csv('year_codes_2021.csv') 

In [285]:
year_codes_2020 = year_codes[year_codes['Year'] == '2020']
year_codes_2020.head(7)

,Year,Code,count
0,2020,10F,146
1,2020,10B,52
2,2020,10C,16
3,2020,08A,16
4,2020,04L,12
5,2020,22C,10
6,2020,04M,5


In [286]:
year_codes_2020.to_csv('year_codes_2020.csv') 